<a href="https://colab.research.google.com/github/humaran27/trabajo_final/blob/main/Trabajo_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import pandas as pd
import gradio as gr
import os

# Cargar CSV
try:
    df = pd.read_csv('peliculas.csv')
    print("CSV cargado correctamente.")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    df = pd.DataFrame(columns=["title", "genre", "consensus", "critic_score", "people_score", "synopsis"])

# Función para buscar películas por género o título
def buscar_pelicula(pregunta):
    pregunta = pregunta.lower()

    # Buscar por género
    for genero in df['genre'].dropna().unique():
        if genero.lower() in pregunta:
            resultado = df[df['genre'].str.lower().str.contains(genero.lower())]
            if not resultado.empty:
                resultado = resultado.sample(1)  # Solo una respuesta aleatoria
            return resultado[['title', 'genre', 'consensus']]

    # Buscar por título
    for titulo in df['title'].dropna().unique():
        if titulo.lower() in pregunta:
            resultado = df[df['title'].str.lower().str.contains(titulo.lower())]
            if not resultado.empty:
                resultado = resultado.sample(1)  # Solo una respuesta aleatoria
            return resultado[['title', 'genre', 'consensus']]

    return "No se encontró ninguna película relacionada con tu pregunta."

# Función para convertir puntajes en estrellas
def puntaje_a_estrellas(puntaje):
    if puntaje >= 90:
        return "⭐️⭐️⭐️⭐️⭐️"
    elif puntaje >= 75:
        return "⭐️⭐️⭐️⭐️"
    elif puntaje >= 60:
        return "⭐️⭐️⭐️"
    elif puntaje >= 40:
        return "⭐️⭐️"
    elif puntaje > 0:
        return "⭐️"
    return "Sin calificación"

# Clasificación por puntajes de estrellas
def clasificacion_estrellas():
    if df.empty:
        return pd.DataFrame([["Sin datos", "", "", ""]], columns=["Título", "Críticos", "Público", "Estrellas"])

    df_filtrado = df[['title', 'critic_score', 'people_score']].dropna()
    df_filtrado['Promedio'] = (df_filtrado['critic_score'] + df_filtrado['people_score']) / 2
    df_filtrado['Estrellas'] = df_filtrado['Promedio'].apply(puntaje_a_estrellas)
    return df_filtrado[['title', 'critic_score', 'people_score', 'Estrellas']].rename(columns={
        "title": "Título", "critic_score": "Críticos", "people_score": "Público"
    })

# Función para analizar y clasificar comentarios
def analizar_comentarios():
    if df.empty:
        return pd.DataFrame([["Sin comentarios", "", 0]], columns=["Título", "Comentario", "Puntuación"])

    comentarios = df[['title', 'consensus']].dropna()
    comentarios['Puntuación'] = comentarios['consensus'].apply(lambda x: puntuar_comentario(x))
    comentarios['Clasificación'] = comentarios['Puntuación'].apply(lambda x: "Positivo" if x > 50 else "Negativo")

    return comentarios[['title', 'consensus', 'Puntuación', 'Clasificación']].rename(columns={
        "title": "Título", "consensus": "Comentario", "Puntuación": "Puntuación (1-100)"
    })

# Función para puntuar el comentario
def puntuar_comentario(comentario):
    comentario = comentario.lower()
    palabras_positivas = ["excelente", "buena", "genial", "fantástica", "increíble", "maravillosa", "disfruté", "recomendable", "impresionante", "entretenida", "sorprendente", "emocionante", "bien hecha"]
    palabras_negativas = ["mala", "horrible", "aburrida", "predecible", "débil", "decepcionante", "pérdida de tiempo", "floja", "terrible", "insufrible", "tonta", "mal escrita"]

    total_positivas = sum(1 for palabra in palabras_positivas if palabra in comentario)
    total_negativas = sum(1 for palabra in palabras_negativas if palabra in comentario)

    if total_positivas + total_negativas == 0:
        return 50  # Neutral si no se encuentran palabras clave

    # Puntaje proporcional entre 0 y 100
    score = (total_positivas / (total_positivas + total_negativas)) * 100
    return int(score)


# GRADIO: Interfaz
with gr.Blocks(theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🎬 Buscador y Calificador de Películas")

    with gr.Tab("🔍 Buscar Película"):
        pregunta = gr.Textbox(label="Haz una pregunta (por género o título)")
        buscar_btn = gr.Button("Buscar")
        resultado_busqueda = gr.DataFrame(label="Resultados")
        buscar_btn.click(fn=buscar_pelicula, inputs=pregunta, outputs=resultado_busqueda)

    with gr.Tab("⭐ Clasificación por Estrellas"):
        calificar_btn = gr.Button("Mostrar Clasificación")
        resultado_calificacion = gr.DataFrame(label="Clasificación por Estrellas")
        calificar_btn.click(fn=clasificacion_estrellas, inputs=[], outputs=resultado_calificacion)

    with gr.Tab("💬 Comentarios de Usuarios"):
        comentarios_btn = gr.Button("Mostrar Comentarios")
        resultado_comentarios = gr.DataFrame(label="Comentarios de Películas")
        comentarios_btn.click(fn=analizar_comentarios, inputs=[], outputs=resultado_comentarios)

app.launch()


CSV cargado correctamente.
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8b655fc1669f3a4ea0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
